In [2]:
import os
import pandas as pd
import requests
import base64

#using azure
API_KEY = "2DWhd5NlWo6U8lpUBSWgyCuCL1bgNUAlQyv4sF2Oq6TsNxszq97NJQQJ99AKACHrzpqXJ3w3AAABACOGzNKu"
ENDPOINT = "https://rg-01-hippa-standalone-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-15-preview"

system_contexts = {
    "location": """
Your task is to determine if the following Mohs patient note describes any tumor type that is both **cancercous** and located on one of the following areas: **Nose, Eyelid, Nail, Lip, Genitalia, or Acral**.
    
ONLY respond with 'Yes' or 'No'.
    
If a tumor type is not explicitly named, use clinical context and wording in the note to make a best-guess classification.
""",
    "aggressive_tumor": """
Your task is to determine if the following Mohs patient note contains any tumor types that qualify as Aggressive Tumor Types.
    
ONLY respond with 'Yes' or 'No'.
    
If tumor type is not explicitly named, use clinical context and wording in the note to make a best-guess classification.
    
Tumor Type Criteria:
- Nonaggressive Tumor Types: Superficial BCC, Nodular BCC, SCCIS, Well-Differentiated SCC
- Aggressive Tumor Types: All other BCC, All other SCC, AFX, Sebaceous Carcinoma, Sweat Gland Carcinoma, Merkel Cell

Note: Aggressive tumor types are less common than nonaggressive ones.  
Only respond 'Yes' if the tumor type name or clinical description **explicitly and clearly** supports an aggressive classification.  
If the note uses vague, uncertain, or general terms — or if strong evidence of aggressiveness is missing — respond 'No'.  
When in doubt, favor 'No'.
""",    
    "dfsp": """
You are a medical assistant trained to identify Dermatofibrosarcoma Protuberans (DFSP) from Mohs patient notes.

Your task is to read the patient note and determine if it indicates a tumor consistent with DFSP.

ONLY respond with 'Yes' or 'No'.

If DFSP is not explicitly mentioned, analyze clinical wording and context to make a best-guess classification.

DFSP Context:
- Dermatofibrosarcoma Protuberans (DFSP) is a rare soft tissue sarcoma that originates in the dermis.
- It often appears as a firm, protuberant plaque or nodule.
- Common colors: pink, reddish-brown, or violet.
- Common locations: the trunk and proximal extremities.
""",
    "recurrent_tumor": """
You are a medical assistant trained to identify whether a tumor is recurrent based on Mohs patient notes.
    
Your task is to read the patient note and determine if it indicates the presence of a recurrent tumor.
    
ONLY respond with 'Yes' or 'No'.
    
If recurrence is not explicitly mentioned, analyze clinical wording and context to make a best-guess classification.
    
Recurrence Context:
- A tumor is considered recurrent only if it is occurring at the site of a prior Mohs surgery or excision (e.g., scar line, previously treated area).
- Tumors at sites previously treated with Curettage and Fulguration (C&F) do NOT qualify as recurrent.
""",
    
    "cf": """
You are identifying notes that reflect a **definitive plan** or completed procedure for **curettage and electrofulguration (C&F)** for treatment of biopsy-confirmed **basal cell carcinoma (BCC)** or **squamous cell carcinoma (SCC)**.

Return “Yes” **only if all three** of the following are true for **at least one lesion**:

---

1. **C&F Mentioned with Intent**
- The note includes the phrase “curettage and electrofulguration” or “curettage and fulguration”
- It appears in the A&P, procedure note, or addendum
- The procedure is **performed** or **firmly planned** with patient election  
- Accept: “patient elects,” “treated with,” “will proceed with,” “scheduled for”  
- Also accept: “tentatively scheduled” **only if** the cancer is confirmed and the plan is clearly part of a coordinated procedure (e.g., same-day as Mohs)

---

2. **Cancer Confirmation**
- The lesion has a **biopsy-confirmed** BCC or SCC
- Pathology may be in the same note or an addendum
- Same-day C&F is only valid **if pathology confirms** cancer and lesion linkage is clear

---

3. **Diagnosis-Treatment Link**
- The C&F treatment applies to a lesion that is clearly linked to the confirmed diagnosis
- Linkage may be via label (e.g., “lesion A”) or anatomical location (e.g., “right dorsal wrist”)

---

Return “No” if:
- C&F was performed **before cancer confirmation** (empirical treatment)
- The treated lesion is **not** BCC or SCC
- C&F is proposed but **not elected** by the patient (e.g., “may consider,” “if needed”)
- The only diagnosis is **actinic keratosis**
- **Mohs or excision** is the primary treatment for the same lesion

Respond only with: **Yes** or **No**
""",
    "ae": """
You are a medical assistant trained to identify if Additional Excision is being planned, recommended, or discussed from Mohs patient notes.

ONLY respond with 'Yes' or 'No'.

If additional excision is not explicitly mentioned, analyze clinical wording and context to make a best-guess classification.

Respond "Yes" only if there is a clear and specific indication that AE is necessary. Acceptable signs include:
- **Direct statements**: e.g., “additional excision planned,” “to return for excision,” “AE scheduled,” or “coordinating additional excision.”
- **Definitive pathology findings** that justify AE:
  - “Positive margin,” “tumor extends to base,” or “not clear at deep margin.”
  - “Residual tumor remains,” “tumor incompletely excised,” or similar definitive phrases.
  - Identification of **aggressive or infiltrative subtypes** **in direct connection with incomplete excision**.
- **Clear plan or communication** to the patient or care team that further surgery is required **due to positive/incomplete margins or tumor characteristics**.

Return "No" if:
- The Addendum confirms complete excision or negative margins with no mention of further surgery.
- Follow-up is for routine care, observation, or cosmetic reasons only.
- There is no explicit or strongly implied plan for further excision.

When uncertain, lean toward "No" unless the note makes a **specific, medically justified case** for AE. Do not guess based on subtle wording alone.
""",
    "ms": """
You are reviewing dermatology notes to determine whether TWO OR MORE distinct lesions are actively planned for Mohs surgery in this clinical encounter.

Respond “Yes” ONLY if ALL of the following conditions are clearly met:

1. **TWO OR MORE BCC/SCC LESIONS**:
   - At least 2 **anatomically distinct lesions**, or clearly separate biopsy sites (e.g., cheek AND temple; Lesion A AND B).
   - Each lesion must be confirmed as **BCC** or **SCC** (ignore AK, melanoma, nevi, or "neoplasm of uncertain behavior").

2. **CURRENT MOHS PLANNING FOR EACH**:
   - Mohs must be explicitly stated in the current visit as the plan for BOTH sites:
     - Phrases like "Mohs scheduled," "Mohs recommended," "Phone Mohs completed," or "plan to do Mohs for both."
     - If a **shared Mohs plan** is described for multiple lesions (e.g., “Mohs same day”), count as YES — if no other treatment is listed.

3. **NO MIXED MANAGEMENT**:
   - If ANY of the lesions is being treated with excision, ED&C, topical treatments (like Efudex), or monitoring — respond “No.”
   - Exception: If Efudex is mentioned for field therapy **but Mohs is still planned and scheduled for specific lesions**, count as “Yes.”

4. **SAME ENCOUNTER, ACTIVE PLAN**:
   - All decisions must occur in this visit — not historical or vague plans.
   - Consent, scheduling, or coordination for Mohs must be clearly documented now.

Clarifications:
- “Mohs for lesions A and B” = Yes.
- “Plan to treat both sites with Mohs” = Yes, unless alternate treatments are listed.
- “Mohs for A, excision for B” = No.
- If biopsy confirmation isn’t explicitly restated but clearly inferred (e.g., Mohs coordination follows a named SCC), accept.

Be strict about treatment type, but flexible about phrasing when intent is clear for 2+ cancers.

Respond ONLY with: Yes or No
""",
    "ent": """
You are a dermatology specialist analyzing clinical notes to determine whether ENT or oculoplastics are currently or will be involved in surgical reconstruction.

Focus specifically on surgical coordination, not just diagnoses or biopsies. Read the **entire note**, especially **Addenda**, as surgical planning often appears there.

Return "Yes" if ANY of the following are present:
- ENT or oculoplastics are **scheduled**, **coordinated**, or **consulted** for repair or reconstruction
- Mohs is planned on the **eyelid**, **medial canthus**, **nasal root**, or **scalp**, AND any mention of coordination, consult, or referral to ENT/oculoplastics is made, even if indirect or in addenda
- ENT or oculoplastics are **mentioned in the context of repair**, including:
   - "ENT reconstruction"
   - "Oculoplastics to assist"
   - "ENT consult placed"
   - "Will coordinate with oculoplastics"
   - "ENT will close"
   - "Mohs with oculoplastics"
   - “Plan to coordinate with ENT or oculoplastics”
   - “Repair options include oculoplastics”
- ANY mention of Mohs with surgical repair AND the note includes **terms like**:
   - “Dr. Powers and ENT”
   - “Message Dr. Shriver”
   - “Will plan same day Mohs”
   - “Addend Mohs order”
   - “In conjunction with oculoplastics”

Return "No" only if:
- ENT/oculoplastics are **not mentioned at all**
- ENT is referenced **only for unrelated issues** (e.g. hearing, congestion)
- The note says "may refer" or "will consider"
- Past coordination is mentioned but **no future intent or planning is described**

**Ambiguous phrases like "to determine role", "discussed repair complexity", or "will coordinate if needed" → count as Yes.**

Your output must be **only**: `Yes` or `No` — no explanations.
""",
    "tumor_size": """
You are a medical assistant trained to extract **tumor size measurements** from Mohs patient notes.

Your task is to identify and return **each distinct tumor size** mentioned in the note, based on available clinical language.

Tumor size should be output as a list of strings, each representing a **2-dimensional measurement** in one of the following formats, ranked from best to fallback:

1. "Acm x Bcm" or "Amm x Bmm" (e.g., "1.2cm x 0.8cm") ← Best format  
2. "A x Bcm" or "A x Bmm" (e.g., "1 x 0.7cm") ← Acceptable fallback  
3. "A x B" (e.g., "0.8 x 0.5") ← Acceptable if no units are available  
4. "Acm" or "Amm" ← Use only if single measurement is given  
5. "A" ← Use only if no units or dimensions are provided

---

**Instructions:**

- Capture **each measurement** mentioned (there is likely more than one in the note).
- If a measurement uses **units** (cm or mm), preserve them.
- If no unit is provided, assume the note is ambiguous and fall back to format #3, #4, or #5 based on available data.
- If the size is given as a range (e.g., "0.7–0.9cm"), extract it as "0.7cm x 0.9cm".
- Ignore references to excision size, defect size, or repair dimensions unless it clearly refers to tumor size.
- Only include **tumor sizes**, not scar, flap, or graft sizes.

---

**Output format:**
A Python-style list of strings. Example:
["1.2cm x 0.8cm", "0.9 x 0.6cm", "0.5", "7mm x 1.4cm"]

Do not add comments, explanations, or anything outside the list.
"""
}

def call_gpt(query, column_name):
    system_context = system_contexts.get(column_name)
    
    headers = {
        "Content-Type": "application/json",
        "api-key": API_KEY,
    }
    
    # Payload for the request
    payload = {
    "messages" : [
            {"role": "system", "content": system_context},
            {"role": "user", "content": f"{query}"},
        ],
      "temperature": 0.3,
      "top_p": 0.95,
      "max_tokens": 30
    }
    
     # Send request
    try:
        response = requests.post(ENDPOINT, headers=headers, json=payload)
        response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
    except requests.RequestException as e:
        print("Response Content:", response.text)
        raise SystemExit(f"Failed to make the request. Error: {e}")

    # Handle the response as needed (e.g., print or process)
    #print(response.json())
    return response.json()["choices"][0]["message"]["content"]
print("Done")

Done


In [3]:
# Use full Excel sheet
import pandas as pd
Cat = pd.read_excel(r'../De-Identified Mohs Full Data.xlsx',sheet_name='Identified Data',header=0) # Only Size at Greatest Dimension column
print("Done")

Done


In [4]:
# Set options here

sample_size = len(Cat) # Change number to increase or decrease number of patients read from file
multiple_columns = False # Set to True to use all input text columns, False to use only PE column
prioritize_column = 2   # Set to 0=PE 1=A&P 2=A to use prioritizeColumn(#), else uses callAll()
                        # If multiple_columns = False, the value of prioritize_column will be the only column used
    
# Change to write GPT responses to a different txt file
txt_file = r'GPT_data/dump_data.txt' # Just dump all responses to dump_data.txt by default

# Array of indexes containing patients with notes that have problems (Some may be ok, these are from size so a location could exist but not size)
skip_patients_array = [7,8,22,23,32,34,38,39,49,50,52,55,62,66,73,80,88,89,101,102,107,110,121,123,131,133,135,
                       136,137,140,150,155,157,164,170,184,187,193,200,206,210,214,227,228,229,234,252,261,265,279,287,292,293]
skip_patients = False # Set to True to skip bad patient notes, False to keep them in

# Changes options with input values
def changeOptions(sample_size_new, multiple_columns_new, prioritize_column_new, txt_file_new, skip_patients_new):
    global sample_size, multiple_columns, prioritize_column, txt_file, skip_patients
    sample_size = sample_size_new
    multiple_columns = multiple_columns_new
    prioritize_column = prioritize_column_new
    txt_file = txt_file_new
    skip_patients = skip_patients_new

bad_index_flag = False # Used for Additional Excision only
print("Done")

# changeOptions() Cases:
# (x, True, 0, x, x) = inputSplit() -> prioritizeColumn(0) -> prioritizeColumnVoteCheck()
#                     All cols used -> call only PE col    -> if PE said 'yes', call other cols to see if either agrees and return yes if so, else no

# (x, True, 8, x, x) = inputSplit() -> callAll()     -> majorityVoteCheck()
#                     All cols used -> call all cols -> check to see if any 2 columns agree and return if so, return Addendum if no matches

# (x, False, 0, x, x) = call_gpt(0)
#                       Single column used, callGPT on that column and use that answer

# (x, False, 5, x, x) = call_gpt(5)
#                       There are only PE, A&P, and A at 0, 1, and 2, so this just gives an error, do not do this


# prioritizeColumnVoteCheck() is used to lower the number of false positives

Done


In [5]:
# Takes the text in the PE, A&P, and A columns and asks ChatGPT to return "Yes" or "No" to the question
# May use only A column
import re

# Declare dictionary variables for later use, indexed by columns
true_results = {}
true_results_final = {}

output = {}
output_final = {}

# Get column name to save GPT column responses to txt file
def getColumnName(column_num):
    column_name = ""
    if column_num == 0: column_name = "PE"
    elif column_num == 1: column_name = "A&P"
    elif column_num == 2: column_name = "A"
    return column_name

# Regex to get only yes/no
def cleanResponse(tmp_ans):
    check_ans = re.split(r'\s+|-', tmp_ans)  # Split by spaces or hyphens
    check_ans = [re.sub(r'[^a-zA-Z]', '', j).lower() for j in check_ans]  # Remove non-alphabetic characters, convert to lowercase
    return [j for j in check_ans if j in ['yes', 'no']] # Keep only "yes" or "no"

# Regex for Size question to only get numerical values
def cleanResponseSize(tmp_ans):
    check_ans = re.split(r'\s+|-', tmp_ans) # Split by spaces or hyphens
    check_ans = [re.sub(r'[^0-9.]', '', j) for j in check_ans] # Remove non numerical values
    return [j for j in check_ans if j.replace('.', '', 1).isdigit()] # Remove empty strings and isolated periods


# Call all 3 columns at once
def callAll(prompts, column_name, log_file):
    tmp_ans = [""] * 3
    check_ans = [""] * 3
    for i in range(3):
        if bad_index_flag == True and i == 0:
            tmp_ans[i] = "NaN error"
            print(f"tmp_ans[{i}] = {str(tmp_ans[i])}")
            check_ans[i] = cleanResponse(tmp_ans[i])
            print(f"check_ans[{i}] = {str(check_ans[i])}")
            
            column_print = getColumnName(i)
            log_file.write(f"{column_print} Unfiltered: {tmp_ans[i]}\n")
            log_file.write(f"{column_print} Filtered: {check_ans[i]}\n")
            continue
            
        tmp_ans[i] = call_gpt(prompts[i], column_name) # Call GPT with prompt and save answer to array
        print(f"tmp_ans[{i}] = {str(tmp_ans[i])}")
        check_ans[i] = cleanResponse(tmp_ans[i])
        print(f"check_ans[{i}] = {str(check_ans[i])}")
        
        column_print = getColumnName(i)
        log_file.write(f"{column_print} Unfiltered: {tmp_ans[i]}\n")
        log_file.write(f"{column_print} Filtered: {check_ans[i]}\n")
    return tmp_ans, check_ans

# Call all 3 columns at once (for Size question only)
def callAllSize(prompts, column_name, log_file):
    tmp_ans = [0] * 3
    check_ans = [0] * 3
    for i in range(3):
        tmp_ans[i] = call_gpt(prompts[i], column_name) # Call GPT with prompt and save answer to array
        print(f"tmp_ans[{i}] = {str(tmp_ans[i])}")
        check_ans[i] = cleanResponseSize(tmp_ans[i])
        print(f"check_ans[{i}] = {str(check_ans[i])}")
        
        column_print = getColumnName(i)
        log_file.write(f"{column_print} Unfiltered: {tmp_ans[i]}\n")
        log_file.write(f"{column_print} Filtered: {check_ans[i]}\n")
    return tmp_ans, check_ans

# Call GPT only for a single column to save time later
def prioritizeColumn(prompts, column_index, column_name, log_file):
    tmp_ans = [""] * 3
    check_ans = [""] * 3
    tmp_ans[column_index] = call_gpt(prompts[column_index], column_name)
    check_ans[column_index] = cleanResponse(tmp_ans[column_index])  
    
    column_print = getColumnName(column_index)
    log_file.write(f"{column_print} Unfiltered: {tmp_ans[column_index]}\n")
    log_file.write(f"{column_print} Filtered: {check_ans[column_index]}\n")
    
    return tmp_ans, check_ans

# Check if 2 arrays are the same output, if so return one of them if it is not empty
def majorityVoteCheck(tmp_ans, check_ans):
    if check_ans[0] == check_ans[1] or check_ans[0] == check_ans[2]:
        if check_ans[0]: return tmp_ans[0]
        elif check_ans[1]: return tmp_ans[1]
        elif check_ans[2]: return tmp_ans[2]
    elif check_ans[1] == check_ans[2]:
        if check_ans[1]: return tmp_ans[1]
        elif check_ans[2]: return tmp_ans[2]
    else:
        return tmp_ans[2]

# Check if 2 arrays are the same output, if so return one of them if it is not empty (only for Size question)
def majorityVoteCheckSize(tmp_ans, check_ans):
    if check_ans[0] == check_ans[1] or check_ans[0] == check_ans[2]:
        if check_ans[0]: return tmp_ans[0]
        elif check_ans[1]: return tmp_ans[1]
        elif check_ans[2]: return tmp_ans[2]
    elif check_ans[1] == check_ans[2]:
        if check_ans[1]: return tmp_ans[1]
        elif check_ans[2]: return tmp_ans[2]
    
    # If no matches, return first numerical string
    for i in range(len(check_ans)):
        if any(j not in ['', '.'] for j in check_ans[i]):
            return tmp_ans[i]
        
    # All non numerical answers, return empty string
    return ""
            
# Only call other 2 columns to check for false positives (0=PE 1=A&P 2=A)
def prioritizeColumnVoteCheck(prompts, tmp_ans, check_ans, column_name, prioritize_column, log_file):
    if "yes" in check_ans[prioritize_column]:
        other_col_indices = [i for i in [0, 1, 2] if i != prioritize_column] # Dynamically get other column indicies
        
        first_other = other_col_indices[0] # Grab 1 of the 2 non prioritized columns
        tmp_ans[first_other] = call_gpt(prompts[first_other], column_name)
        print(f"tmp_ans[{first_other}] = {tmp_ans[first_other]}")
        check_ans[first_other] = cleanResponse(tmp_ans[first_other])
        print(f"check_ans[{first_other}] = {check_ans[first_other]}")
        
        column_print = getColumnName(first_other)
        log_file.write(f"{column_print} Unfiltered: {tmp_ans[first_other]}\n")
        log_file.write(f"{column_print} Filtered: {check_ans[first_other]}\n")
        
        if "yes" in check_ans[first_other]: # If yes in column then it agrees, return yes
            return "yes"
        
        second_other = other_col_indices[1] # No match in 1st grabbed column, call the other
        tmp_ans[second_other] = call_gpt(prompts[second_other], column_name)
        print(f"tmp_ans[{second_other}] = {tmp_ans[second_other]}")
        check_ans[second_other] = cleanResponse(tmp_ans[second_other])
        print(f"check_ans[{second_other}] = {check_ans[second_other]}")
        
        column_print = getColumnName(second_other)
        log_file.write(f"{column_print} Unfiltered: {tmp_ans[second_other]}\n")
        log_file.write(f"{column_print} Filtered: {check_ans[second_other]}\n")
        
        if "yes" in check_ans[second_other]: # If yes in column then it agrees, return yes
            return "yes"
        else:
            return "no" # Prioritized column said Yes, but others disagreed, possible false positive
    else: return "no"
        
    
# Always split into 3 GPT calls, one for each column
def inputSplit(input_columns, que, index, prioritize_column, column_name, log_file):
    while len(input_columns) < 3: # Some text columns may be empty so add a blank entry
        input_columns.append("")
        
    input_string = " ".join(map(str, input_columns)) # Apply str() to each index of input_columns then join with a space
    prompt = f"{que} \n\n {input_string}" # Create base prompt (question + input_columns)
    log_file.write(f"Index: {index}\n") # Write index to file
    
    if(len(input_string) > 0):
        prompts = [f"{que} \n\n {col}" for col in input_columns] # Recreate prompt with question + 1 column
        
        if prioritize_column in [0, 1, 2]: # PE, A&P, A
            tmp_ans, check_ans = prioritizeColumn(prompts, prioritize_column, column_name, log_file)
            return prioritizeColumnVoteCheck(prompts, tmp_ans, check_ans, column_name, prioritize_column, log_file)
        elif "tumor size" in que.lower():
            tmp_ans, check_ans = callAllSize(prompts, column_name, log_file)
            return majorityVoteCheckSize(tmp_ans, check_ans)
        else:
            tmp_ans, check_ans = callAll(prompts, column_name, log_file)
            return majorityVoteCheck(tmp_ans, check_ans)
        
        # --------------The rest of this function below is currently not used, might help but probably won't--------------
        
        # Count num of "yes" and "no"
        yes_count = [0] * len(input_columns)
        no_count = [0] * len(input_columns)
        
        for i in range(len(check_ans)):
            yes_count[i] = check_ans[i].count('yes')
            no_count[i] = check_ans[i].count('no')
            
        print(f"Yes counts: {yes_count}")
        print(f"No counts: {no_count}")
        
        # If no matches, return string that has the most "yes" or "no"
        if any(count > 0 for count in yes_count) or any(count > 0 for count in no_count): # Check count strings for any occurences or "yes" or "no"
            max_yes_no = [0] * len(input_columns)
            max_answer = 0
            max_i = -1
            
            # Check if string has more "yes" or "no"
            for i in range(len(input_columns)):
                max_yes_no[i] = max(yes_count[i], no_count[i])
                
            # Look for highest overall "yes" or "no" then return that string
            for i in range(len(input_columns)):
                if max_answer < max_yes_no[i]:
                    max_answer = max_yes_no[i]
                    max_i = i
            return tmp_ans[max_i]
        
        # All non "yes" or "no" answers, return empty string
        return ""
    else:
        # Somehow input_string is empty
        return "input_string empty"

def runColumn(column_name, result_column_string, que_new):
    n = min(sample_size, len(Cat)) # Get number of input notes
    
    true_results[column_name] = [0]*n
    true_results_final[column_name] = [0]*n
    
    output[column_name] = [0]*n
    output_final[column_name] = [0]*n

    with open(txt_file, "a") as log_file: # Open file to write column answers to for vote testing
        for i in range(0, n):
            # If skip_patients is True and current index is in skip_patients_array, move to next index
            if skip_patients and i in skip_patients_array:
                continue

            true_results[column_name][i] = Cat.at[i, result_column_string]
            que = que_new
            ans = ""
            
            global bad_index_flag
            
            if i == 98:
                #bad_index_flag = True # Only uncomment this if running Additional Excision
                pass
            else:
                bad_index_flag = False

            if multiple_columns:
                input_columns = Cat.loc[i, ['Physical Exam (PE)', 'Assessment & Plan (A&P)', 'Addendum (A)']] # Select all 3 input columns
                ans = inputSplit(input_columns, que, i, prioritize_column, column_name, log_file) # Result from GPT call
            elif not multiple_columns:
                column_names = ['Physical Exam (PE)', 'Assessment & Plan (A&P)', 'Addendum (A)']
                input_columns = Cat.at[i, column_names[prioritize_column]] # Select only the chosen input column, not all 3
                prompt = f"{que} \n\n {input_columns}"
                
                ans = call_gpt(prompt, column_name) # Call GPT
                #print(f"tmp_ans[{i}] = {ans}")
                ans_clean = cleanResponse(ans) # Regex clean GPT's response
                #print(f"check_ans[{i}] = {ans_clean}")

                column_name_short = getColumnName(prioritize_column) # Get selected column name to dynamically save data to the txt_file
                log_file.write(f"Index: {i}\n") # Write index to file
                log_file.write(f"{column_name_short} Unfiltered: {ans}\n")
                log_file.write(f"{column_name_short} Filtered: {ans_clean}\n")

            print(f"Result[{i}] = {ans}")
            output[column_name][i] = ans # Save cleaned GPT response for later use
print("Done")

Done


In [13]:
# Change options for current column, then run the column, below are the parameters for changeOptions() then runColumn()
# sample_size, multiple_columns, prioritize_column, txt_file, skip_patients
# column_name, result_column_string, sample_size, que

# r'GPT_data/garbage_code_testing_data.txt' for testing
changeOptions(10, True, 2, r'GPT_data_final/location_data_final.txt', False) # Location
runColumn("location", 'Location - Nose, Eyelid, Nail, Lip, Genitalia, or Acral (L)', 'Question: Does the following patient note contain a cancerous tumor on the: Nose, Eyelid, Nail, Lip, Genitalia, or Acral?')

tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[0] = No
tmp_ans[0] = Yes
check_ans[0] = ['yes']
tmp_ans[1] = Yes
check_ans[1] = ['yes']
tmp_ans[2] = Yes
check_ans[2] = ['yes']
Result[1] = Yes
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[2] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[3] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[4] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[5] = No
tmp_ans[0] = Yes
check_ans[0] = ['yes']
tmp_ans[1] = Yes
check_ans[1] = ['yes']
tmp_ans[2] = Yes
check_ans[2] = ['yes']
Result[6] = Yes
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans

In [5]:
changeOptions(10, False, 2, r'GPT_data_final/aggressive_tumor_data_final.txt', False) # Aggressive Tumor
runColumn("aggressive_tumor", 'Aggressive Tumor Pathology Type (AT)', 'Question: Does the following patient note contain an Aggressive Tumor Type?')

tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[0] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = Yes
check_ans[2] = ['yes']
Result[1] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[2] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[3] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[4] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[5] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[6] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
che

In [10]:
changeOptions(10, False, 2, r'GPT_data_final/dfsp_data_final.txt', False) # DFSP
runColumn("dfsp", 'DFSP (D)', 'Question: Does the following patient note describe a tumor consistent with Dermatofibrosarcoma Protuberans (DFSP)?')

tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[0] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[1] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[2] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[3] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[4] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[5] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[6] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check

In [15]:
changeOptions(10, True, 2, r'GPT_data_final/recurrent_tumor_data_final.txt', False) # Recurrent Tumor
runColumn("recurrent_tumor", 'Recurrent Tumor (R)', 'Question: Does the following patient note indicate that any of the tumors are recurrent?')

tmp_ans[0] = Yes
check_ans[0] = ['yes']
tmp_ans[1] = Yes
check_ans[1] = ['yes']
tmp_ans[2] = Yes
check_ans[2] = ['yes']
Result[0] = Yes
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = Yes
check_ans[1] = ['yes']
tmp_ans[2] = Yes
check_ans[2] = ['yes']
Result[1] = Yes
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[2] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[3] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[4] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[5] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[6] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2

In [5]:
changeOptions(10, True, 7, r'GPT_data/cf_data_final.txt', False) # Additional C&F
runColumn("cf", 'Additional C&F', 'Does this note contain a CURRENT, CONFIRMED plan for curettage & electrofulguration (C&F/ED&C) to treat biopsy-proven skin cancer?')

tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[0] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[1] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[2] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[3] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = Yes
check_ans[2] = ['yes']
Result[4] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[5] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[6] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
che

In [74]:
changeOptions(10, False, 2, r'GPT_data/ae_data(3).txt', False) # Additional Excision
runColumn("ae", 'Additional Excision', 'Does this case require an additional excision?')
# Uncomment bad_index_flag for this question
# 'Question: Does this case require an additional excision?' gets flagged more than 'Does this case require an additional excision?' somehow???

Result[0] = Yes
Result[1] = Yes
Result[2] = No
Result[3] = No
Result[4] = Yes
Result[5] = No
Result[6] = No
Result[7] = Yes
Result[8] = No
Result[9] = Yes
Result[10] = Yes
Result[11] = Yes
Result[12] = No
Result[13] = No
Result[14] = Yes
Result[15] = No
Result[16] = No
Result[17] = No
Result[18] = Yes
Result[19] = Yes
Result[20] = Yes
Result[21] = Yes
Result[22] = Yes
Result[23] = No
Result[24] = Yes
Result[25] = No
Result[26] = Yes
Result[27] = No
Result[28] = Yes
Result[29] = Yes
Result[30] = No
Result[31] = No
Result[32] = No
Result[33] = Yes
Result[34] = Yes
Result[35] = Yes
Result[36] = No
Result[37] = No
Result[38] = Yes
Result[39] = Yes
Result[40] = No
Result[41] = No
Result[42] = Yes
Result[43] = Yes
Result[44] = Yes
Result[45] = Yes
Result[46] = Yes
Result[47] = Yes
Result[48] = No
Result[49] = No
Result[50] = Yes
Result[51] = Yes
Result[52] = Yes
Result[53] = Yes
Result[54] = No
Result[55] = Yes
Result[56] = Yes
Result[57] = Yes
Result[58] = No
Result[59] = Yes
Result[60] = Y

In [7]:
changeOptions(10, True, 7, r'GPT_data/ms_data_final.txt', False) # Multiple Sites
runColumn("ms", 'Multiple Sites', 'Question: Are two or more biopsy-confirmed lesions in this note planned for surgical treatment in this visit?')

tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[0] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[1] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[2] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[3] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[4] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[5] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[6] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check

In [8]:
changeOptions(10, True, 7, r'GPT_data/ent_data_final.txt', False) # Coordinated Repair with ENT/Oculoplastics
runColumn("ent", 'Coordinated repair with ENT/oculoplastics (C)', 'Question: Does this note indicate current or planned coordination with ENT or oculoplastics for surgical repair/reconstruction?')

tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[0] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[1] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[2] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = Yes
check_ans[1] = ['yes']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[3] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[4] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[5] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
check_ans[2] = ['no']
Result[6] = No
tmp_ans[0] = No
check_ans[0] = ['no']
tmp_ans[1] = No
check_ans[1] = ['no']
tmp_ans[2] = No
che

In [6]:
changeOptions(len(Cat), True, 7, r'GPT_data/tumor_size_data.txt', False) # Size at Greatest Dimension
runColumn("tumor_size", 'Size at Greatest Dimension (S) ***Be  sure this is from the note and not the Mohs episode!', 'Question: What are the tumor sizes listed in the following patient note?')

tmp_ans[0] = ["0.8 x 0.8 cm", "0.4 x 0.5 cm"]
check_ans[0] = ['0.8', '0.8', '0.4', '0.5']
tmp_ans[1] = []
check_ans[1] = []
tmp_ans[2] = ["0.8 x 0.8 cm", "0.4 x 0.5 cm"]
check_ans[2] = ['0.8', '0.8', '0.4', '0.5']
Result[0] = ["0.8 x 0.8 cm", "0.4 x 0.5 cm"]
tmp_ans[0] = ["0.6cm x 0.4cm"]
check_ans[0] = ['0.6', '0.4']
tmp_ans[1] = ["0.6cm x 0.4cm"]
check_ans[1] = ['0.6', '0.4']
tmp_ans[2] = ["0.7 x 0.7cm"]
check_ans[2] = ['0.7', '0.7']
Result[1] = ["0.6cm x 0.4cm"]
tmp_ans[0] = ["3mm x 3mm"]
check_ans[0] = ['3', '3']
tmp_ans[1] = []
check_ans[1] = []
tmp_ans[2] = []
check_ans[2] = []
Result[2] = ["3mm x 3mm"]
tmp_ans[0] = ["6.5 x 5.0cm"]
check_ans[0] = ['6.5', '5.0']
tmp_ans[1] = ["3.0"]
check_ans[1] = ['3.0']
tmp_ans[2] = []
check_ans[2] = []
Result[3] = ["6.5 x 5.0cm"]
tmp_ans[0] = ["0.6 x 0.8cm", "1.2 x 1.6cm", "1 x 1cm", "
check_ans[0] = ['0.6', '0.8', '1.2', '1.6', '1', '1']
tmp_ans[1] = ["1cm"]
check_ans[1] = ['1']
tmp_ans[2] = []
check_ans[2] = []
Result[4] = ["0.6 x 0.8cm", "1.

In [14]:
# Go through every GPT response and turn it into "Y" or "N" to compare
def cleanFinal(column_name):
    n = len(output[column_name])
    for i in range(0, n):
        if skip_patients and i in skip_patients_array:
            continue

        yn = output[column_name][i]
        print(f"i={i} Answer: {yn}")
        yn_real = str(true_results[column_name][i])

        yn = re.split(r'\s+|-', yn)  # Split by spaces or hyphens
        yn = [re.sub(r'[^a-zA-Z]', '', j).lower() for j in yn]  # Remove non-alphabetic characters, convert to lowercase
        yn = [j for j in yn if j in ['yes', 'no']] # Keep only "yes" or "no"

        if len(yn) == 0:
            output_final[column_name][i] = "error_2"  # No valid response detected, bad prediction

        yes_count = yn.count('yes')
        no_count = yn.count('no')

        if yes_count > no_count:
            output_final[column_name][i] = "Y"
        elif no_count > yes_count:
            output_final[column_name][i] = "N"

        # Check if the given input is valid
        if yn_real.strip() and yn_real.lower() != 'nan':
            if 'y' in yn_real.lower():
                true_results_final[column_name][i] = "Y" # Some are input like N/Y or Y/N/N so look for Y and if exists compare with that
            else:
                true_results_final[column_name][i] = yn_real.strip()
        else:
            true_results_final[column_name][i] = "error_1" # No valid input provided in true_results, bad note

        print(f"Result[{i}]: ChatGPT Prediction = {output_final[column_name][i]} Answer Given = {true_results_final[column_name][i]}")
    print("-"*40)
    
# Cleans up GPT responses and Ground Truth for later comparisons
#cleanFinal("location")
cleanFinal("aggressive_tumor")
cleanFinal("dfsp")
cleanFinal("recurrent_tumor")

i=0 Answer: No
Result[0]: ChatGPT Prediction = N Answer Given = N
i=1 Answer: Yes
Result[1]: ChatGPT Prediction = Y Answer Given = Y
i=2 Answer: No
Result[2]: ChatGPT Prediction = N Answer Given = N
i=3 Answer: No
Result[3]: ChatGPT Prediction = N Answer Given = N
i=4 Answer: No
Result[4]: ChatGPT Prediction = N Answer Given = N
i=5 Answer: No
Result[5]: ChatGPT Prediction = N Answer Given = N
i=6 Answer: Yes
Result[6]: ChatGPT Prediction = Y Answer Given = Y
i=7 Answer: No
Result[7]: ChatGPT Prediction = N Answer Given = N
i=8 Answer: No
Result[8]: ChatGPT Prediction = N Answer Given = N
i=9 Answer: No
Result[9]: ChatGPT Prediction = N Answer Given = N
i=10 Answer: No
Result[10]: ChatGPT Prediction = N Answer Given = N
i=11 Answer: No
Result[11]: ChatGPT Prediction = N Answer Given = N
i=12 Answer: No
Result[12]: ChatGPT Prediction = N Answer Given = N
i=13 Answer: Yes
Result[13]: ChatGPT Prediction = Y Answer Given = Y
i=14 Answer: No
Result[14]: ChatGPT Prediction = N Answer Given =

In [15]:
def confusionMatrix(column_name):
    TP = TN = FP = FN = 0
    n = len(output[column_name])
    for i in range(0, n):
        prediction = output_final[column_name][i]
        truth = true_results_final[column_name][i]
        
        if truth not in ("Y", "N"): # Skip any invalid Ground Truths
            continue
        
        if prediction == truth: # True Positive/True Negative
            if prediction == "Y": # True Positive
                TP += 1
            elif prediction == "N": # True Negative
                TN += 1
        elif prediction != truth: # False Positive/False Negative
            if prediction == "Y": # False Positive
                FP += 1
            elif prediction == "N": # False Negative
                FN += 1
    return {"TP": TP, "TN": TN, "FP": FP, "FN": FN}

def confusionMetrics(confusion_matrix):
    TP = confusion_matrix["TP"]
    TN = confusion_matrix["TN"]
    FP = confusion_matrix["FP"]
    FN = confusion_matrix["FN"]
    
    total = TP + TN + FP + FN
    accuracy = (TP + TN) / total if total else 0 # How often did model predict correctly
    precision = TP / (TP + FP) if (TP + FP) else 0 # When model predicted Y, how often was it correct
    recall = TP / (TP + FN) if (TP + FN) else 0 # How many of the true Y cases did model correctly identify
    F1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0 # Balance between Precision and Recall
    
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "F1": F1}

def printConfusion(column_name):
    matrix = confusionMatrix(column_name)
    metrics = confusionMetrics(matrix)
    
    print(f"Confusion Matrix for {column_name}:")
    for name, value in matrix.items():
        print(f"{name}: {value}")

    print(f"\nConfusion Matrix Metrics for {column_name}:")
    for name, value in metrics.items():
        print(f"{name}: {(value * 100):.2f}")
    print("\n")

#printConfusion("location")
printConfusion("aggressive_tumor")
printConfusion("dfsp")
printConfusion("recurrent_tumor")

Confusion Matrix for location:
TP: 75
TN: 181
FP: 25
FN: 10

Confusion Matrix Metrics for location:
accuracy: 87.97
precision: 75.00
recall: 88.24
F1: 81.08


